In [ ]:
pip install transformers peft accelerate trl bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 131.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 185.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 130.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 200.2 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.8/899.8 MB 109.5 MB/s  0:00:060:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 169.5 MB/s  0:00:020:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 186.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 202.2 MB/s  0:00:00m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 184.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 163.6 MB/s  0:00:030:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 202.9 MB/s  0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 230.9 MB/s  0:0

In [ ]:
pip install seaborn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 116.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 160.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 153.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 162.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [seaborn]m6/8 [matplotlib]
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
import torch.nn as nn
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import IA3Config, get_peft_model, TaskType
from tqdm import tqdm
from matplotlib import pyplot as plt

In [ ]:
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, prepare_model_for_kbit_training
from trl import SFTTrainer

In [ ]:
from huggingface_hub import login
login()

In [ ]:
quantization_config = transformers.BitsAndBytesConfig(load_in_4bit=True)

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mixtral-8x7B-Instruct-v0.1", truncation=True, padding=True, padding_side="right")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mixtral-8x7B-Instruct-v0.1", quantization_config=quantization_config)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model = prepare_model_for_kbit_training(model)

config = LoraConfig(r = 16,
                    lora_alpha=16,
                    target_modules = ["gate", "q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
                    lora_dropout=0.1
                    )



lora_model = get_peft_model(model, config)

lora_model.print_trainable_parameters()

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

...

model-00018-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00019-of-00019.safetensors:   0%|          | 0.00/4.22G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

trainable params: 15,732,736 || all params: 46,718,525,440 || trainable%: 0.0337


In [ ]:
import bitsandbytes as bnb

def get_gate_weight_4bit(gate):
    # gate: Linear4bit(in_features=2048, out_features=60, bias=False)

    if hasattr(gate, "base_layer"):
        gate = gate.base_layer

    qweight = gate.weight.data
    qweight = qweight.view(-1)

    W = bnb.functional.dequantize_4bit(
        qweight,
        quant_state=gate.quant_state,
        quant_type="nf4",
    )
    W = W.view(gate.out_features, gate.in_features)
    return W  # float16/float32 matrix, shape == (60, 2048)

def get_router_l2_norms(model):
    layer_norms = {}

    for name, module in model.named_modules():
        if name.endswith("block_sparse_moe.gate"):
            print(f"name: {name} module: {module}")
            weight = module.weight
            print(f"weights {weight.shape}")
            weight_dequantization = get_gate_weight_4bit(module)
            print(f"weight after dequantization {weight_dequantization.shape}")
           # norms = torch.norm(weight_dequantization.float(), dim=1)  # shape (60,)
            #print(f" norms {norms.shape}")
            #layer_norms[name] = norms.detach().cpu()



            # Case 2: normal FP16/FP32 Linear router

            # Compute per-expert L2 norms
            #norms = torch.norm(W.float(), dim=1)  # shape (8,)
            #print(f"Router L2 norms: {norms.tolist()}")

            #layer_norms[name] = norms.detach().cpu()

    return layer_norms


norms = get_router_l2_norms(lora_model)


name: base_model.model.model.layers.0.block_sparse_moe.gate module: lora.Linear4bit(
  (base_layer): Linear4bit(in_features=4096, out_features=8, bias=False)
  (lora_dropout): ModuleDict(
    (default): Dropout(p=0.1, inplace=False)
  )
  (lora_A): ModuleDict(
    (default): Linear(in_features=4096, out_features=16, bias=False)
  )
  (lora_B): ModuleDict(
    (default): Linear(in_features=16, out_features=8, bias=False)
  )
  (lora_embedding_A): ParameterDict()
  (lora_embedding_B): ParameterDict()
  (lora_magnitude_vector): ModuleDict()
)
weights torch.Size([16384, 1])
weight after dequantization torch.Size([8, 4096])
name: base_model.model.model.layers.1.block_sparse_moe.gate module: lora.Linear4bit(
  (base_layer): Linear4bit(in_features=4096, out_features=8, bias=False)
  (lora_dropout): ModuleDict(
    (default): Dropout(p=0.1, inplace=False)
  )
  (lora_A): ModuleDict(
    (default): Linear(in_features=4096, out_features=16, bias=False)
  )
  (lora_B): ModuleDict(
    (default): 

In [ ]:
import re

def extract_router_modules(model):
    routers = {}  # { layer_index: gate_module }
    for name, module in model.named_modules():
        if name.endswith("block_sparse_moe.gate"):
            layer = int(re.search(r"layers\.(\d+)\.", name).group(1))
            routers[layer] = module
    return routers


def get_W_from_gate(gate):
    # unwrap lora
    if hasattr(gate, "base_layer"):
        gate = gate.base_layer

    # quantized case
    if hasattr(gate, "quant_state"):
        q = gate.weight.data.view(-1)
        W = bnb.functional.dequantize_4bit(
            q,
            quant_state=gate.quant_state,
            quant_type="nf4"
        ).view(gate.out_features, gate.in_features)
    else:
        W = gate.weight.data

    return W.float()   # (8, 4096)


# --------------------------
# 2. Extract LoRA A and B for all router layers
# --------------------------
def extract_router_lora_AB(model):
    A_dict, B_dict = {}, {}

    for name, param in model.named_parameters():
        if "block_sparse_moe.gate.lora_A" in name:
            layer = int(re.search(r"layers\.(\d+)\.", name).group(1))
            A_dict[layer] = param.detach().float()
        elif "block_sparse_moe.gate.lora_B" in name:
            layer = int(re.search(r"layers\.(\d+)\.", name).group(1))
            B_dict[layer] = param.detach().float()

    return A_dict, B_dict

# --------------------------
# 3. Compute W, AB, and W+AB norms
# --------------------------
def compute_router_norms(model, alpha=4, r=4):
    scale = alpha / r

    routers = extract_router_modules(model)
    A_dict, B_dict = extract_router_lora_AB(model)

    norms_W = {}
    norms_AB = {}
    norms_WAB = {}

    for layer, gate in routers.items():

        # ----- W -----
        W = get_W_from_gate(gate)        # (8, 4096)
        norms_W[layer] = torch.norm(W, dim=1).tolist()

        # ----- AB -----
        if layer in A_dict:   # LoRA exists
            A = A_dict[layer]     # (rank, 4096)
            B = B_dict[layer]     # (8, rank)
            delta = scale * (B @ A)  # (8, 4096)
        else:
            delta = torch.zeros_like(W)

        norms_AB[layer] = torch.norm(delta, dim=1).tolist()

        # ----- W + AB -----
        merged = W + delta
        norms_WAB[layer] = torch.norm(merged, dim=1).tolist()

    return norms_W, norms_AB, norms_WAB

norm_W, norm_AB, norm_WAB = compute_router_norms(lora_model)

In [ ]:
from trl import SFTConfig
import logging
logging.getLogger("trl.trainer.sft_trainer").setLevel(logging.ERROR)

In [ ]:
dataset = load_dataset("Na0s/sft-ready-Text-Generation-Augmented-Data", split="train")

README.md:   0%|          | 0.00/344 [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

data/train-00000-of-00022.parquet:   0%|          | 0.00/332M [00:00<?, ?B/s]

data/train-00001-of-00022.parquet:   0%|          | 0.00/323M [00:00<?, ?B/s]

...

data/train-00019-of-00022.parquet:   0%|          | 0.00/119M [00:00<?, ?B/s]

data/train-00020-of-00022.parquet:   0%|          | 0.00/98.5M [00:00<?, ?B/s]

data/train-00021-of-00022.parquet:   0%|          | 0.00/109M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7667416 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/22 [00:00<?, ?it/s]

In [ ]:

trainer = SFTTrainer(
    model = lora_model,
    train_dataset = dataset,
    processing_class = tokenizer,
    args = SFTConfig(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 16,
        packing = True,
        group_by_length = True,
        warmup_steps = 5,
        bf16 = True,
        max_steps=1000,
        learning_rate = 2e-4,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        eval_strategy="no",
        do_eval=False,
        output_dir = "./outputs",
        push_to_hub=True,
        remove_unused_columns=False,
    )
)
tokenized_dataset = trainer.train_dataset
tokenized_dataset.save_to_disk("tokenized_packed_mixtral_dataset")
trainer.train()

Adding EOS to train dataset:   0%|          | 0/7667416 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/7667416 [00:00<?, ? examples/s]

Packing train dataset:   0%|          | 0/7667416 [00:00<?, ? examples/s]

Saving the dataset (0/61 shards):   0%|          | 0/2491572 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 32000}.


Step,Training Loss
10,0.887700
20,0.810800
30,0.776700
40,0.849100
50,0.737500
60,0.835300
70,0.788200
80,0.790700
90,0.749900
100,0.718400


TrainOutput(global_step=1000, training_loss=0.7489152665138245, metrics={'train_runtime': 36581.9995, 'train_samples_per_second': 0.437, 'train_steps_per_second': 0.027, 'total_flos': 4.5376709815780147e+18, 'train_loss': 0.7489152665138245, 'epoch': 0.006421648661969231})

In [ ]:
lora_model.save_pretrained("saved_lora_adapter")

In [ ]:
print(norm_WAB)

{0: [2.29744553565979, 2.434562921524048, 2.315584659576416, 2.138937473297119, 2.306791305541992, 2.150372266769409, 2.479288339614868, 1.9992676973342896], 1: [2.0431435108184814, 2.1275172233581543, 1.99368155002594, 1.4405370950698853, 1.914389729499817, 2.099679708480835, 2.1274726390838623, 2.046830177307129], 2: [1.664521336555481, 1.5987746715545654, 1.6822432279586792, 1.6340171098709106, 1.688033103942871, 1.7258613109588623, 1.785442590713501, 1.7724609375], 3: [1.397202491760254, 1.5089452266693115, 1.5188285112380981, 1.562635064125061, 1.543931007385254, 1.5004613399505615, 1.498637080192566, 1.5126264095306396], 4: [1.206209421157837, 1.211559534072876, 1.1849654912948608, 1.2032876014709473, 1.1436771154403687, 1.2141910791397095, 1.2332735061645508, 1.1471960544586182], 5: [1.0494977235794067, 1.000311255455017, 1.0306810140609741, 1.0619970560073853, 0.9863646626472473, 1.056584358215332, 1.0974808931350708, 1.0247920751571655], 6: [0.9021797180175781, 1.0010763406753

In [ ]:
norm_W_after, norm_AB_after, norm_WAB_after = compute_router_norms(lora_model)

In [ ]:
def compute_norm_difference(norm_before, norm_after):
    diff = {}
    for layer in norm_before:
        diff[layer] = [
            abs(norm_after[layer][i] - norm_before[layer][i])
            for i in range(len(norm_before[layer]))
        ]
    return diff

def get_min_diff_indices(norm_diff):
    min_indices = {}
    for layer, diffs in norm_diff.items():
        min_idx = int(torch.tensor(diffs).argmin().item())
        min_indices[layer] = min_idx
    return min_indices

norm_diff = compute_norm_difference(norm_WAB_before, norm_WAB_after)
min_diff_index = get_min_diff_indices(norm_diff)
print(min_diff_index)

matrix = np.zeros((32, 8))  # 32 × 8

for layer, norms in norm_diff.items():
    matrix[layer, :] = norms


plt.figure(figsize=(12, 6))
sns.heatmap(
    matrix,
    cmap="viridis",
    cbar_kws={"label": "L2 Norm Difference"},
    xticklabels=[f"E{i}" for i in range(0, 8)],
    yticklabels=[f"L{i}" for i in range(0, 32)],
)
plt.xlabel("Router Column (Expert)")
plt.ylabel("Layer")
plt.title("L2 Norm Difference per Router Column (Expert) after Fine-Tuning, r=16, lora_alpha=16")
plt.tight_layout()
plt.savefig("heatmap_mixtral_r16.png", dpi=300)